# Enter Dogs vs Cats

## Introduction

This notebook is a template for submission to a Kaggle competition: Dogs vs. Cats Redux: Kernels Edition

To start, we need to get the data ready. First, download the dataset using `kaggle cli`, and then arrange the data into the correct structure like the following:

```
utils/
    vgg16.py
    utils.py
redux.ipynb
data/
    redux/
        train/
            cat.123.jpg
            dog.2334.jpg
        test/
            432.jpg
            222.jpg
```

You can download the data files from the competition page [here](https://www.kaggle.com/c/dogs-vs-cats-redux-kernels-edition/data) or you can download them from the command line using the [Kaggle CLI](https://github.com/floydwch/kaggle-cli).

```
pip install kaggle-cli
kg download -u yingchi.pei@gmail.com -p FL199473/kag -c dogs-vs-cats-redux-kernels-edition
```

In [1]:
# Verify the directory you are in 
%pwd

'/Users/PeiYingchi/Documents/fastai-notes/deeplearning1/nbs'

In [3]:
# Create references to important directories 
import sys, os
current_dir = os.getcwd()
HOME_DIR = current_dir
DATA_HOME_DIR = current_dir + '/data/redux'

In [4]:
from utils import *
from vgg16 import Vgg16

%matplotlib inline

Using Theano backend.


## Action Plan
1. Create validation and sample sets
2. Rearrange image files into their respective folders
3. Finetune and train model
4. Generate predictions
5. Validate predictions
6. Submit predictions to Kaggle

## Create validation and sample set

Before moving on, we need to make sure that we have the `train.zip`, `test.zip` downloaded inside our DATA_HOME_DIR.
For example, inside the `/data` folder, my file structure looks like this:

```
yingchi nbs/data ‹master*› » tree -L 2                                                          
.
├── dogscats
│   ├── sample
│   ├── test1
│   ├── train
│   └── valid
└── redux
    ├── sample_submission.csv
    ├── test.zip
    └── train.zip
```